# CDTIME

A big chunk of dealing with CMIP data lay in dealing with each model's different way of representing time

For example let's look at a few model time calendar and base units

In [ ]:
import glob
ta_files = glob.glob("/global/cscratch1/sd/cmip6/CMIP6/CMIP/*/*/historical/r1i1p1f1/Amon/ta/gn/*/*")

In [2]:
import cdms2
models = set()
institutions = set()
for name in ta_files:
    sp = name.split("/")
    institution = sp[7]
    institutions.add(institution)
    model = sp[8]
    if not model in models:
        print("Institution/Model: {}/{}".format(institution, model))
        f = cdms2.open(name)
        ta = f["ta"]
        tim = ta.getTime()
        print("\tTime units:", tim.units)
        print("\tTime calendar:", tim.getCalendar(), tim.calendar)
        f.close()
    models.add(model)

Institution/Model: MRI/MRI-ESM2-0
	Time units: days since 1850-01-01
	Time calendar: 4369 proleptic_gregorian
Institution/Model: NCAR/CESM2-WACCM
	Time units: days since 0001-01-01 00:00:00
	Time calendar: 4113 noleap
Institution/Model: NCAR/CESM2
	Time units: days since 0001-01-01 00:00:00
	Time calendar: 4113 noleap
Institution/Model: NASA-GISS/GISS-E2-1-H
	Time units: days since 1850-1-1
	Time calendar: 4113 365_day
Institution/Model: NASA-GISS/GISS-E2-1-G
	Time units: days since 1850-1-1
	Time calendar: 4113 365_day
Institution/Model: MIROC/MIROC6
	Time units: days since 1850-1-1
	Time calendar: 135441 gregorian
Institution/Model: CAMS/CAMS-CSM1-0
	Time units: days since 1865-1-1 00:00
	Time calendar: 4113 365_day
Institution/Model: SNU/SAM0-UNICON
	Time units: days since 1850-01-01 00:00:00
	Time calendar: 4113 noleap
Institution/Model: BCC/BCC-ESM1
	Time units: days since 1850-01-01
	Time calendar: 4113 365_day
Institution/Model: BCC/BCC-CSM2-MR
	Time units: days since 1850-01-01

As you can see not all models have the same base time OR the same model.

While cdms can handle this for you, there ight be time where you need to do the math yourself.

The `cdtime` module helps you with this

In [3]:
import cdtime

`cdtime` introduces two types of time objects, ***component*** time object, which are essentially *date* in the format **YYYY-MM-DD HH:mm:SS** and ***relative*** time object, after are a *delta* from some basetime in some defined *units*

## Component Time

***component*** time are very similar to datetime object:

In [4]:
from datetime import datetime
'{:%Y-%m-%d %H:%M:%S}'.format(datetime(2014, 7, 24, 10, 5, 15))

'2014-07-24 10:05:15'

In [5]:
comp = cdtime.componenttime(2014,7,24,10,5, 15)
print(comp)
print(comp.year, comp.month, comp.day, comp.hour, comp.minute, comp.second)

2014-7-24 10:5:15.0
2014 7 24 10 5 15.0


## Relative time

relative time need a *base date* and a *delta* with *units* from it. It is very convenient to convert values in time axes.

In [6]:
rel = cdtime.relativetime(24, "days since 2014-7")
print(rel)
print(rel.value, rel.units)

24.000000 days since 2014-7
24.0 days since 2014-7


## Manipulating cdtime objects


### Comparison

all cdtime time object can be used with each other and compared

In [7]:
base = cdtime.comptime(2014, 7, 1)
before = cdtime.reltime(3, "days since 2014-6")
after = cdtime.reltime(3, "days since 2014-7")
same = cdtime.reltime(0, "days since 2014-7")
print(after.cmp(base))  # Greater
print(before.cmp(base)) # lesss
print(same.cmp(base)) # equal

1
-1
0


### Addition/Subtraction

In [8]:
print(base.add(6, cdtime.Days))

2014-7-7 0:0:0.0


In [9]:
print(base.sub(3, cdtime.Months))

2014-4-1 0:0:0.0


One can specify a calendar to use when doing the addition

In [10]:
print("Regular: ",base.sub(3000,cdtime.Days, cdtime.DefaultCalendar))
print("No Leap:", base.sub(3000,cdtime.Days, cdtime.NoLeapCalendar))

Regular:  2006-4-14 0:0:0.0
No Leap: 2006-4-12 0:0:0.0


### Convertion between types

In [11]:
print(base.torelative("days since 2000"))

5295.000000 days since 2000


In [12]:
print(base.torelative("days since 2000", cdtime.NoLeapCalendar))

5291.000000 days since 2000


# CDMS Time Axes

In [13]:
time = cdms2.createAxis([1,2,3,4,5,6])
time.units = "days since 2016-02-27"
time.designateTime()
time.setBounds(None)

print(time.asRelativeTime("days since 2000"))
print(time.asComponentTime())
print(time.asComponentTime(cdtime.NoLeapCalendar))

[5902.000000 days since 2000, 5903.000000 days since 2000, 5904.000000 days since 2000, 5905.000000 days since 2000, 5906.000000 days since 2000, 5907.000000 days since 2000]
[2016-2-28 0:0:0.0, 2016-2-29 0:0:0.0, 2016-3-1 0:0:0.0, 2016-3-2 0:0:0.0, 2016-3-3 0:0:0.0, 2016-3-4 0:0:0.0]
[2016-2-28 0:0:0.0, 2016-3-1 0:0:0.0, 2016-3-2 0:0:0.0, 2016-3-3 0:0:0.0, 2016-3-4 0:0:0.0, 2016-3-5 0:0:0.0]


## Changing units

In [14]:
tim2 = time.clone()  # To preserve the original
tim2.toRelativeTime("days since 2000")
print("Reg:", tim2[:])
tim2.toRelativeTime("days since 2000", cdtime.NoLeapCalendar)
print("No Leap:", tim2[:])


Reg: [5902 5903 5904 5905 5906 5907]
No Leap: [5898 5899 5899 5900 5901 5902]
